## Set up

In [54]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
sns.set_style("darkgrid")
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [49]:
# Windy grid
rewardSize = -1
gamma = 0.95 # discount rate

alpha = 0.85 # learning rate
epsilon = 0.9 # e-greedy 
episodeNum = 100 # number of episodes 
maxSteps = 100

In [50]:
import gym
import gym_gridworlds
from lib.envs.windy_gridworld import WindyGridworldEnv

env = WindyGridworldEnv()

### Utilities

In [51]:
#Function to choose the next action 
def choose_action(state, Q): 
    action=None
    if np.random.uniform(0, 1) < epsilon: 
        action = env.action_space.sample()
    else: 
        action = np.argmax(Q[state, :]) 
    return action 

def createEpsilonGreedyPolicy(Q, epsilon, num_actions): 
    """ 
    Creates an epsilon-greedy policy based 
    on a given Q-function and epsilon. 
       
    Returns a function that takes the state 
    as an input and returns the probabilities 
    for each action in the form of a numpy array  
    of length of the action space(set of possible actions). 
    """
    def policyFunction(state): 
   
        Action_probabilities = np.ones(num_actions, 
                dtype = float) * epsilon / num_actions 
                  
        best_action = np.argmax(Q[state]) 
        Action_probabilities[best_action] += (1.0 - epsilon) 
        return Action_probabilities 
   
    return policyFunction 

## Monte Carlo

![title](img/montecarlo.png)

In [40]:
def Monte_Carlo():
    #Initializing the Q-matrix and Returns matrix
    policy = createEpsilonGreedyPolicy(Q, epsilon, env.action_space.n)
    
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)
    
    for i in range(episodeNum):
        episode = [] # current episode
        state = env.reset()
        for t in range(maxSteps):
            action_probs = policy(state)
            action = np.random.choice(np.arange(len(action_probs)), p = action_probs) 
            
            next_state, reward, done, _ = env.step(action)
            episode.append((state, action, reward))
            if done:
                break
            state = next_state
            
            sa_in_episode = set([(tuple(x[0]), x[1]) for x in episode])
            
    for state, action in sa_in_episode:
        sa_pair = (state, action)
        first_occur_index = next(i for i, x in enumerate(episode) if (x[0] == state and x[1] == action))
        G = np.sum([])
                
            

## SARSA(0)

In [52]:
def SARSA_0():
    #Initializing the Q-matrix
    Q = np.zeros((env.observation_space.n, env.action_space.n)) 
    #Initializing the reward  
    reward=0

    # Starting the SARSA learning 
    for episode in range(episodeNum): 
        t = 0
        state1 = env.reset()
        action1 = choose_action(state1, Q) 


        while t < maxSteps: 
            #Getting the next state 
            state2, reward, done, info = env.step(action1) 

            #Choosing the next action 
            action2 = choose_action(state2, Q) 
            
            #Learning the Q-value 
            predict = Q[state1, action1] 
            target = reward + gamma * Q[state2, action2] 
            Q[state1, action1] = Q[state1, action1] + alpha * (target - predict)

            state1 = state2 
            action1 = action2 

            #Updating the respective vaLues 
            t += 1
            reward += rewardSize

            #If at the end of learning process 
            if done: 
                break

    return Q
SARSA_0()

array([[-18.33574913, -17.42376719, -17.56174589, -17.63852137],
       [-18.24509149, -18.77059763, -15.19689868, -18.15352496],
       [-18.70443   , -18.58418828, -18.72741887, -18.46015189],
       [-18.90201731, -18.98671894, -18.91727076, -18.54381478],
       [-18.89179623, -18.99896306, -18.83359513, -18.91551547],
       [-18.72341235, -18.92186913, -18.95285582, -18.98546935],
       [-17.50273806, -18.99231009, -18.69686597, -18.99133498],
       [-18.38582386, -14.2696141 , -18.53097014, -18.77284933],
       [-17.23789216, -11.69580801, -17.10007082, -17.73836444],
       [-16.51125096, -16.24819263, -10.68012384, -10.07629561],
       [-18.45955432, -18.15493956, -16.2899608 , -18.15996949],
       [-18.21915125, -18.4583135 , -17.78521901, -18.19330664],
       [-18.58296151, -18.16145592, -18.36392481, -17.13538854],
       [-18.78187765, -18.48353909, -18.54863664, -18.43628347],
       [-18.5069888 , -18.55071849, -16.15224826, -18.92744865],
       [-18.47427044, -18

## SARSA(n)

In [28]:
#Function to learn the Q-value 
def SARSA_update(state, state_n, reward, action, action_n): 
    predict = Q[state, action] 
    target = reward + gamma * Q[state2, action2] 
    
    Q[state, action] = Q[state, action] + alpha * (target - predict) 
    
def SARSA_update(state1, state_n, n, action, action_n):
    predict = Q[state1, action1]
    target = gamma**n * Q[state_n, action_n]
    n -= 1
    for i in range(n):
        state_new, reward, done, info = env.step(action1)
        action_new = choose_action(state_new)
        action1 = action_new
        target += gamma**i * reward
        
    Q[state, action] = Q[state, action] + alpha * (target - predict) 

![title](https://miro.medium.com/max/1400/1*-v5wbqLYCvzrOQE2Zmv05g.png)

In [38]:
def SARSA_n(n):
    Q = np.zeros((env.observation_space.n, env.action_space.n)) 
    #Initializing the reward  
    total_reward=0
    max_reward = 0

    # Starting the SARSA learning 
    for episode in range(episodeNum): 
        t = 0
        tau = 0
        T = sys.maxsize
        stored_actions = {}
        stored_rewards = {}
        stored_states = {}
        reward_episode = 0
        
        stored_states[0] = state = env.reset()
        stored_actions[0] = choose_action(state, Q)
        
        
        while (tau < T - 1):
            if (t < T):
                action_t = choose_action(state=stored_states[t % n], Q=Q)
                stored_states[(t+1) % n], stored_rewards[(t+1) % n], done, info = env.step(action_t) 
                state_t = stored_states[(t+1) % n]
                
                reward_episode += stored_rewards[(t+1) % n]
                total_reward += stored_rewards[(t+1) % n]
                
                if (done):
                    T = t + 1
                else:
                    stored_actions[(t+1) % n] = choose_action(stored_states[t % n], Q)
                
                    
            tau = t - n + 1
            G = 0
            if (tau >= 0):
                for i in range(tau+1, min(tau+n, T)+1):
                    G += gamma ** (i - tau - 1) * stored_rewards[i % n] 
                if (tau + n < T):
                    G += gamma ** n * Q[stored_states[(tau + n) % n]][stored_actions[(tau + n) % n]]
                current_Q = Q[stored_states[tau % n]][stored_actions[tau % n]]
                Q[stored_states[tau % n]][stored_actions[tau % n]] += alpha * (G - current_Q)
                
            t += 1
                
        max_reward = max(reward_episode, max_reward)
        
    return Q, max_reward, total_reward/episodeNum
                    

In [39]:
Q, max_reward, average_reward = SARSA_n(10)
print(Q)
print(max_reward)
print(average_reward)

[[-20.         -20.         -20.         -20.        ]
 [-20.         -20.         -20.         -20.        ]
 [-20.         -20.         -20.         -20.        ]
 [-20.         -20.         -20.         -20.        ]
 [-20.         -20.         -20.         -20.        ]
 [-20.         -20.         -20.         -20.        ]
 [-20.         -20.         -20.         -20.        ]
 [-20.         -20.         -20.         -20.        ]
 [-20.         -20.         -20.         -20.        ]
 [-19.99999643 -19.9999986  -19.99999661 -20.        ]
 [-20.         -20.         -20.         -20.        ]
 [-20.         -20.         -20.         -20.        ]
 [-20.         -20.         -20.         -20.        ]
 [-20.         -20.         -20.         -20.        ]
 [-20.         -20.         -20.         -20.        ]
 [-20.         -20.         -20.         -20.        ]
 [-18.92133064 -19.69139777 -19.53165902 -18.92133064]
 [-20.         -20.         -20.         -20.        ]
 [-20.    

## Q-leanring

![title](https://www.cse.unsw.edu.au/~cs9417ml/RL1/images/qalg.gif)

In [55]:
def Q_learning():
    Q = np.zeros((env.observation_space.n, env.action_space.n)) 
    policy = createEpsilonGreedyPolicy(Q, epsilon, env.action_space.n)
    for episde in range(episodeNum):
        state = env.reset()
        t = 0
        while (t < maxSteps):
            action_probs = policy(state)
            # choose action according to  
            # the probability distribution 
            action = np.random.choice(np.arange(len(action_probs)), p = action_probs) 
            
            next_state, reward, done, info = env.step(action)
            
            next_best_action = np.argmax(Q[next_state, :])
            target = reward + gamma * Q[next_state, next_best_action]
            Q[state, action] += alpha * (target - Q[state, action])
            
            if (done):
                break
            
            state = next_state
            t += 1